In [1]:
!rm -f hmp.parquet*

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200116040724-0000
KERNEL_ID = 705fbdbe-b142-4466-9658-4ac65ea634fd


In [2]:
!wget https://github.com/IBM/coursera/raw/master/hmp.parquet

--2020-01-16 04:07:28--  https://github.com/IBM/coursera/raw/master/hmp.parquet
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/coursera/master/hmp.parquet [following]
--2020-01-16 04:07:28--  https://raw.githubusercontent.com/IBM/coursera/master/hmp.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 932997 (911K) [application/octet-stream]
Saving to: 'hmp.parquet'

100%[======================================>] 932,997     --.-K/s   in 0.03s   

2020-01-16 04:07:29 (30.8 MB/s) - 'hmp.parquet' saved [932997/932997]



In [3]:
df = spark.read.parquet('hmp.parquet')
df.createOrReplaceTempView('df')
df.show()

+---+---+---+--------------------+-----------+
|  x|  y|  z|              source|      class|
+---+---+---+--------------------+-----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|
| 20| 50| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 50| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 35|Accelerometer-201...|Brush_teeth|
| 21| 51| 33|Accelerometer-201...|Brush_teeth|
| 20| 50| 34|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 20| 51| 35|Accelerometer-201...|Brush_teeth|
| 18| 49| 34|Accelerometer-201...|Brush_teeth|
| 19| 48| 34|Accelerometer-201...|Brush_teeth|
| 16| 53| 34|Accelerometer-201...|Brush_teeth|
| 18| 52| 35|

In [4]:
splits = df.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [8]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer

indexer = StringIndexer(inputCol="class", outputCol="label")
encoder = OneHotEncoder(inputCol="label", outputCol="labelVec")
vectorAssembler = VectorAssembler(inputCols=["x","y","z"], outputCol="features")
normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)

In [10]:
from pyspark.ml.classification import LinearSVC

lsvc = LinearSVC(maxIter=10, regParam=0.1)

In [17]:
df.createOrReplaceTempView('df')
df_two_class = spark.sql("""
    select * from df where class in ('Use_telephone','Standup_chair')
""")

In [18]:
splits = df_two_class.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [19]:
df_train.show()

+---+---+---+--------------------+-------------+
|  x|  y|  z|              source|        class|
+---+---+---+--------------------+-------------+
|  0| 30| 24|Accelerometer-201...|Standup_chair|
|  0| 31| 30|Accelerometer-201...|Standup_chair|
|  0| 31| 32|Accelerometer-201...|Standup_chair|
|  0| 31| 32|Accelerometer-201...|Standup_chair|
|  0| 35| 28|Accelerometer-201...|Standup_chair|
|  0| 37| 26|Accelerometer-201...|Standup_chair|
|  0| 37| 35|Accelerometer-201...|Standup_chair|
|  0| 38| 37|Accelerometer-201...|Standup_chair|
|  0| 40| 34|Accelerometer-201...|Standup_chair|
|  0| 42| 34|Accelerometer-201...|Standup_chair|
|  0| 43| 34|Accelerometer-201...|Standup_chair|
|  0| 43| 34|Accelerometer-201...|Standup_chair|
|  0| 44| 21|Accelerometer-201...|Standup_chair|
|  0| 56| 30|Accelerometer-201...|Standup_chair|
|  1| 25| 29|Accelerometer-201...|Standup_chair|
|  1| 27| 20|Accelerometer-201...|Standup_chair|
|  1| 29| 23|Accelerometer-201...|Standup_chair|
|  1| 29| 27|Acceler

In [20]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[indexer, encoder, vectorAssembler, normalizer, lsvc])

In [21]:
model = pipeline.fit(df_train)

In [22]:
prediction = model.transform(df_train)

Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "/opt/ibm/spark/python/pyspark/ml/wrapper.py", line 105, in __del__
    SparkContext._active_spark_context._gateway.detach(self._java_obj)
AttributeError: 'LinearSVC' object has no attribute '_java_obj'


In [23]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(prediction)

0.9392041026607255

In [24]:
prediction = model.transform(df_test)

In [25]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(prediction)

0.9371816338170507